First, we import the required libraries that will help us read and preprocess the data, as well as train the model

In [1]:
from sklearn.model_selection import *
import numpy as np
import pandas as pd

Next, we use a built in pandas function to read in our data and add in column identifiers

In [2]:
names = ['id number', 'clump thickness', 'uniformity of cell size','uniformity of cell shape', 'marignal adhesion'
         , 'single epithelial cell size', 'bare nuclei', 'bland chromatin', 'normal nucleoli', 'mitoses', 'class']

data = pd.read_csv('breast-cancer-wisconsin.data', names = names)

Now, lets print the first five rows of the table to see that the columns are properly identified

In [3]:
print(data.head())

   id number  clump thickness  uniformity of cell size  \
0    1000025                5                        1   
1    1002945                5                        4   
2    1015425                3                        1   
3    1016277                6                        8   
4    1017023                4                        1   

   uniformity of cell shape  marignal adhesion  single epithelial cell size  \
0                         1                  1                            2   
1                         4                  5                            7   
2                         1                  1                            2   
3                         8                  1                            3   
4                         1                  3                            2   

  bare nuclei  bland chromatin  normal nucleoli  mitoses  class  
0           1                3                1        1      2  
1          10                3              

Now, we must do some preprocessing on the data, which includes adding in a bias column, and splitting the data set into the "class" portion, which the predictions will be made on, and the features portion, which includes the rest of the data set minus the id number. Additionally, we fill in all rows that do not contain a numerical value with 1s. I experimented with using the mean and median for this metric as well, but 1s worked the best in terms of accuracy

In [4]:
data['bias'] = 1 # bias column

y = data['class']  
y = y.replace(4, True) # Malignant = true
y = y.replace(2, False) # Benign = false
y = y.values.reshape(len(y), 1) # Data must reshaped into one-dimensional format

n_features = 9
columns = data.columns.tolist()
columns.remove("id number") 
columns.remove("class")  
X = data[columns] 
X = (X.replace('?', np.NaN)).astype(float) # Fill ? entries with NaN, and cast values to floats 
X = X.fillna(0).values # Fill all NaN values with 0s and fetch all values

Now, we implement the sigmoid function from the provided slides

In [5]:
def sigmoid(theta, x):
    z = np.dot(x, theta)
    return (1.0 / (1 + np.exp(-z)))

Similarily, we implement the cross entropy loss function, the gradient, and the hessian based on the notation in the provided slides

In [6]:
def lossfunction(y, mu):
    return -np.sum(y * np.log(mu) + (1 - y) * np.log(1 - mu))

def gradient(x, y, mu):
    return np.dot(x.T, mu - y)

def hessian(x, mu):
    return np.dot((mu * (1 - mu) * x).T, x)

Then, we implement the Newton's Method for logistic regression, also using the slides as a general guideline. We use a very small value, 0.0000000001, to denote the epsilon that is required to make sure that convergence is properly defined. Also note that my model has a slight variation, from the slides. Instead of calling sigmoid to find the mu in the previous functions, I've done this only once in the newton function and passed in to the rest of the functions as a parameter in order to preserve space. The end result is the same.

In [7]:
def newton(X_train, y_train):
    theta = np.array([[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]]) # zeros array the size of n_features + 1
    err = np.Infinity
    Δ = np.Infinity
    while abs(Δ) > 0.0000000001: # Small value to define when convergence occurs
        mu_train = sigmoid(theta, X_train)
        g = gradient(X_train, y_train, mu_train)
        h = hessian(X_train, mu_train)
        theta = theta - np.dot(np.linalg.inv(h), g)
        errorupdate = lossfunction(y_train, mu_train)
        Δ = err - errorupdate
        err = errorupdate      
    return theta

Now, we must implement the accuracy measurement function. To do this, I implemented the classification (mu > 0.5) portion from the course slides, and then divided the number of properly classified elements by the total number of elements to obtain the accuracy score. 

In [8]:
def accuracy(y, mu):
    mu = mu > 0.5 
    x = (y == mu) # Correctly classified
    totlength = len(mu) # Total number of classifications
    return np.sum(x)/totlength # Num correct divided by num total

Finally, we split our data into training and testing sets using scikit learn, call the Newton's Method classifier on it, and record the accuracy. Please note that 10 random train-test splits were performed with a 0.8-0.2 train-test ratio, and the accuracy scores were averaged across all 10 runs. 

In [9]:
accuracies = [] # initialize accuracy array to store the accuracies from 10 splits
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True) # 10 splits, 0.8 - 0.2 train-test ratio
    theta = newton(X_train, y_train) # Run the logistic regression classifier
    mu_test = sigmoid(theta, X_test) # Predict
    accuracies.append(accuracy(y_test, mu_test)) #store each accuracy in array  
avgaccuracy = np.mean(np.array(accuracies)) # average of all accuracies
print('Accuracy: ') 
print(round(avgaccuracy,4)) # accuracy rounded to 4 digits

Accuracy: 
0.9586
